# Modeling and predicting sequential data requires a different approach from standard regression or classification. Luckily, a particular type of Neural Networks called Recurrent Neural Networks (RNNs) are specifically designed for that purpose.

# •	Many-to-one — using a sequence of values to predict the next value. You can find a Python example of this type of setup in my RNN

# In this example we will use Sydney to make a different model doing a sequence of Many-to-one 12-1. 

In [1]:
#Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, SimpleRNN # for creating regular densely-connected NN layers and RNN layers

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import mean_squared_error # for model evaluation metrics
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

C:\Users\Admin\anacondalastone\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Tensorflow/Keras: 2.7.0
pandas: 1.3.4
numpy: 1.26.1
sklearn: 1.2.2
plotly: 4.14.3


In [2]:
# Set Pandas options to display more columns
pd.options.display.max_columns=50

# Read in the weather data csv
df=pd.read_csv('C:/Users/Admin/Machine Learning chapter 5/Deep Learning basics with Python/Recurrent Neural Networks/weatherAUS.csv', encoding='utf-8')

# Drop records where target MinTemp=NaN or MaxTemp=NaN
df=df[pd.isnull(df['MinTemp'])==False]
df=df[pd.isnull(df['MaxTemp'])==False]

# Median daily temperature (mid point between Daily Max and Daily Min)
df['MedTemp']=df[['MinTemp', 'MaxTemp']].median(axis=1)

# Show a snaphsot of data
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,MedTemp
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No,18.15
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No,16.25
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No,19.30
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No,18.60
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No,24.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,ESE,E,15.0,13.0,59.0,27.0,1024.7,1021.2,NaN,NaN,9.4,20.9,No,No,12.65
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No,13.10
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No,14.45
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No,16.15


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143579 entries, 0 to 145458
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           143579 non-null  object 
 1   Location       143579 non-null  object 
 2   MinTemp        143579 non-null  float64
 3   MaxTemp        143579 non-null  float64
 4   Rainfall       141330 non-null  float64
 5   Evaporation    81916 non-null   float64
 6   Sunshine       74919 non-null   float64
 7   WindGustDir    134336 non-null  object 
 8   WindGustSpeed  134397 non-null  float64
 9   WindDir9am     133824 non-null  object 
 10  WindDir3pm     140086 non-null  object 
 11  WindSpeed9am   142472 non-null  float64
 12  WindSpeed3pm   141206 non-null  float64
 13  Humidity9am    142130 non-null  float64
 14  Humidity3pm    140312 non-null  float64
 15  Pressure9am    129902 non-null  float64
 16  Pressure3pm    129908 non-null  float64
 17  Cloud9am       89068 non-null

In [7]:
df['Location'].value_counts()

Canberra            3428
Sydney              3339
Hobart              3192
Perth               3192
Darwin              3191
Adelaide            3189
Brisbane            3172
Cairns              3039
Ballarat            3038
AliceSprings        3037
Townsville          3037
Tuggeranong         3034
MountGambier        3034
Bendigo             3034
GoldCoast           3031
Launceston          3030
Albury              3021
Wollongong          3021
Mildura             3009
PerthAirport        3009
MelbourneAirport    3009
WaggaWagga          3009
SydneyAirport       3008
Moree               3007
NorfolkIsland       3007
Sale                3007
Williamtown         3004
Cobar               3003
Woomera             3002
Watsonia            3002
Penrith             3001
Portland            2999
Witchcliffe         2996
Nuriootpa           2989
CoffsHarbour        2987
Richmond            2986
PearceRAAF          2972
NorahHead           2967
BadgerysCreek       2967
Walpole             2954


# Given the data contains weather information for multiple locations across Australia, let’s pick one city (Canberra) and plot the daily median temperature on a chart.

In [80]:
# Select only Canberra 
dfSydney=df[df['Location']=='Sydney'].copy()

# Plot daily median temperatures in Canberra
fig = go.Figure()
fig.add_trace(go.Scatter(x= dfSydney['Date'], 
                         y= dfSydney['MedTemp'],
                         mode='lines',
                         name='Median Temperature',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Date'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Median Daily Temperatures in Sydney", 
                             font=dict(color='black')))

fig.show()

In [10]:
#Select data for modeling and apply MinMax scaling
X= dfSydney[['MedTemp']]
scaler = MinMaxScaler()
X_scaled=scaler.fit_transform(X)


#Create training and testing samples
train_data, test_data = train_test_split(X_scaled, test_size=0.2, shuffle=False)

In [11]:
print(train_data.shape)
print(test_data.shape)

(2671, 1)
(668, 1)


In [12]:
#We will use this function in the next step to get the data into the right shape
def prep_data(datain, time_step):
    # 1. y-array  
    # First, create an array with indices for y elements based on the chosen time_step
    y_indices = np.arange(start=time_step, stop=len(datain), step=time_step)
    # Create y array based on the above indices 
    y_tmp = datain[y_indices]
    
    # 2. X-array  
    # We want to have the same number of rows for X as we do for y
    rows_X = len(y_tmp)
    # Since the last element in y_tmp may not be the last element of the datain, 
    # let's ensure that X array stops with the last y
    X_tmp = datain[range(time_step*rows_X)]
    # Now take this array and reshape it into the desired shape
    X_tmp = np.reshape(X_tmp, (rows_X, time_step, 1))
    return X_tmp, y_tmp

Let's Break down what it's going on in the above code. I will use the train data to explain the code. 

y is the target data.
after we will use the following paramters: 
time_step = 12
X_train, y_train = prep_data(train_data, time_step)
X_test, y_test = prep_data(test_data, time_step)

In [13]:
time_step = 12

In [14]:
y_indices =  np.arange(start=time_step, stop=len(train_data), step=time_step)

In [15]:
#we have the indices of the target data that it's every 7 time steps. 
y_indices

array([  12,   24,   36,   48,   60,   72,   84,   96,  108,  120,  132,
        144,  156,  168,  180,  192,  204,  216,  228,  240,  252,  264,
        276,  288,  300,  312,  324,  336,  348,  360,  372,  384,  396,
        408,  420,  432,  444,  456,  468,  480,  492,  504,  516,  528,
        540,  552,  564,  576,  588,  600,  612,  624,  636,  648,  660,
        672,  684,  696,  708,  720,  732,  744,  756,  768,  780,  792,
        804,  816,  828,  840,  852,  864,  876,  888,  900,  912,  924,
        936,  948,  960,  972,  984,  996, 1008, 1020, 1032, 1044, 1056,
       1068, 1080, 1092, 1104, 1116, 1128, 1140, 1152, 1164, 1176, 1188,
       1200, 1212, 1224, 1236, 1248, 1260, 1272, 1284, 1296, 1308, 1320,
       1332, 1344, 1356, 1368, 1380, 1392, 1404, 1416, 1428, 1440, 1452,
       1464, 1476, 1488, 1500, 1512, 1524, 1536, 1548, 1560, 1572, 1584,
       1596, 1608, 1620, 1632, 1644, 1656, 1668, 1680, 1692, 1704, 1716,
       1728, 1740, 1752, 1764, 1776, 1788, 1800, 18

In [17]:
train_data.shape, y_indices.shape

((2671, 1), (222,))

2671 values divided in 222 arrays of 12 values each. 

The training data has a total of 2671 values and the last target index is in value 2664. 2664 + 12 = 2676, exceed the last index value for the training data. Into training data the values in the index (2665,2671),6 values, are not going to be trained because do not reach the sequence of 12 values. 

In [18]:
#these are the median temperatures every 7 days. 
y_tmp = train_data[y_indices]
y_tmp

array([[0.45995893],
       [0.53182752],
       [0.46201232],
       [0.59548255],
       [0.38603696],
       [0.39014374],
       [0.39425051],
       [0.35318275],
       [0.25462012],
       [0.24845996],
       [0.34907598],
       [0.19712526],
       [0.16221766],
       [0.16632444],
       [0.0862423 ],
       [0.08418891],
       [0.06776181],
       [0.24229979],
       [0.27310062],
       [0.69609856],
       [0.31416838],
       [0.11088296],
       [0.58932238],
       [0.46817248],
       [0.42505133],
       [0.50308008],
       [0.3613963 ],
       [0.4661191 ],
       [0.65092402],
       [0.59137577],
       [0.74948665],
       [0.58521561],
       [0.59342916],
       [0.57084189],
       [0.51950719],
       [0.33675565],
       [0.3798768 ],
       [0.28952772],
       [0.28336756],
       [0.35728953],
       [0.24024641],
       [0.1724846 ],
       [0.37166324],
       [0.20944559],
       [0.15195072],
       [0.17864476],
       [0.18685832],
       [0.295

In [19]:
#every 12 days we have a total of 222 values. 
y_tmp.shape

(222, 1)

In [20]:
# We want to have the same number of rows for X as we do for y
rows_X = len(y_tmp)

In [21]:
rows_X

222

In [23]:
train_data.shape

(2671, 1)

In [24]:
X_tmp = train_data[range(time_step*rows_X)]

In [27]:
time_step*rows_X

2664

In [25]:
X_tmp 

array([[0.47433265],
       [0.54004107],
       [0.56057495],
       ...,
       [0.17043121],
       [0.14784394],
       [0.17043121]])

In [26]:
#222*12 = 2664. This data is flatten. 
X_tmp.shape

(2664, 1)

In [28]:
train_data[0], X_tmp[0]

(array([0.47433265]), array([0.47433265]))

As mentoned earlier the last values of the train_data are not going to be considered for training since they do not reach the minimumn sequence lenght, 12. 

In [29]:
# Now take this array and reshape it into the desired shape
X_tmp = np.reshape(X_tmp, (rows_X, time_step, 1))

In [30]:
X_tmp.shape

(222, 12, 1)

# With the reshape we can see that for each y_target temperature,222 in total, we have 12 previous median temperature that predict the target value.  

In [31]:
#we have 222 target temperatures that are predicted using the 12 previous median day temperatures. 
y_tmp.shape

(222, 1)

In [32]:
#sanme number of rows but with 12 previous median Temperature compared to the target y. 
X_tmp.shape

(222, 12, 1)

In [34]:
X_train, y_train = prep_data(train_data, time_step)
X_test, y_test = prep_data(test_data, time_step)

In [35]:
#X_train are the sequence values and y_train is the target value. 
X_train.shape, y_train.shape

((222, 12, 1), (222, 1))

In [36]:
X_test.shape , y_test.shape

((55, 12, 1), (55, 1))

In [37]:
X_train[0], y_train[0]

(array([[0.47433265],
        [0.54004107],
        [0.56057495],
        [0.49691992],
        [0.54620123],
        [0.58726899],
        [0.53593429],
        [0.42505133],
        [0.37782341],
        [0.41067762],
        [0.44147844],
        [0.56262834]]),
 array([0.45995893]))

# Output Layer uses activation linear because it is a reggression model, continuos data. An every 12 median temperatures(input data) we predict the 13th day median temperature. One sequence of 12 values input at a time, predicting one output. 

# Specify the structure of a Neural Network

In [39]:
model = Sequential(name="First-RNN-Model") # Model
model.add(Input(shape=(time_step,1), name='Input-Layer')) # Input Layer - need to specify the shape of inputs
model.add(SimpleRNN(units=10, activation='tanh', name='Hidden-Recurrent-Layer')) # Hidden Recurrent Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
model.add(Dense(units=5, activation='tanh', name='Hidden-Layer')) # Hidden Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
model.add(Dense(units=1, activation='linear', name='Output-Layer')) # Output Layer, Linear(x) = x. one putput because it it many inputs to one output and since it is continous data we use activation = 'linear'

# Compile keras model

In [40]:
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )

# Fit keras model on the dataset

In [41]:
model.fit(X_train, # input data
          y_train, # target data
          batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
          epochs=30, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
          verbose='auto', # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
          callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
          validation_split=0.0, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
          #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
          shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
          class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
          sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
          initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
          steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
          validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
          validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
          validation_freq=1, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
          max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
          workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
          use_multiprocessing=False, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
         )
#since the size is one we have the same number of steps as the X_train and y_train: 222. 

Epoch 1/30
222/222 [==============================] - 2s 3ms/step - loss: 0.0183 - mean_squared_error: 0.0183 - mean_absolute_error: 0.0991
Epoch 2/30
222/222 [==============================] - 1s 4ms/step - loss: 0.0120 - mean_squared_error: 0.0120 - mean_absolute_error: 0.0838
Epoch 3/30
222/222 [==============================] - 1s 4ms/step - loss: 0.0117 - mean_squared_error: 0.0117 - mean_absolute_error: 0.0800
Epoch 4/30
222/222 [==============================] - 1s 3ms/step - loss: 0.0102 - mean_squared_error: 0.0102 - mean_absolute_error: 0.0742
Epoch 5/30
222/222 [==============================] - 1s 4ms/step - loss: 0.0100 - mean_squared_error: 0.0100 - mean_absolute_error: 0.0748
Epoch 6/30
222/222 [==============================] - 1s 4ms/step - loss: 0.0100 - mean_squared_error: 0.0100 - mean_absolute_error: 0.0726
Epoch 7/30
222/222 [==============================] - 1s 4ms/step - loss: 0.0099 - mean_squared_error: 0.0099 - mean_absolute_error: 0.0733
Epoch 8/30
222/222 [

# Use model to make predictions

In [85]:
# Predict the result on training data
pred_train = model.predict(X_train)
# Predict the result on test data
pred_test = model.predict(X_test)

In [86]:
X_train.shape, pred_train.shape

((222, 12, 1), (222, 1))

In [87]:
pred_test.shape, X_test.shape

((55, 1), (55, 12, 1))

In [88]:
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")


-------------------- Model Summary --------------------
Model: "First-RNN-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-Recurrent-Layer (Sim  (None, 10)               120       
 pleRNN)                                                         
                                                                 
 Hidden-Layer (Dense)        (None, 5)                 55        
                                                                 
 Output-Layer (Dense)        (None, 1)                 6         
                                                                 
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________



In your cases this means the trainable parameters are:
recurrent_weights = 10#10 
num_features = 1#10 
biases = 10
100 + 10 + 10 = 120. 

Hidden-Layer (Dense) = 10 * 5 = 50 weights + 5 biases = 55 params
Output-Layer (Dense) = 5 * 1 = 5 weights + 1 biases = 6 params. 

In [58]:
print('-------------------- Weights and Biases --------------------')
print("Note, the last parameter in each layer is bias while the rest are weights")
print("")
for layer in model.layers:
    print(layer.name)
    for item in layer.get_weights():
        print("  ", item)

-------------------- Weights and Biases --------------------
Note, the last parameter in each layer is bias while the rest are weights

Hidden-Recurrent-Layer
   [[-0.45230618 -0.6893798  -0.5023037  -0.3326956  -0.08152603 -0.27449638
   0.02289328 -0.41930264 -0.11217286  0.37457752]]
   [[ 0.18251738  0.45370173 -0.01989795 -0.20980595 -0.45113763  0.09714559
  -0.25271487  0.37297928  0.39144963  0.07358813]
 [-0.13073505 -0.55410975 -0.39140412 -0.28812948  0.02448436  0.27851057
   0.03286833  0.23536886 -0.02020606  0.34258407]
 [ 0.52674353  0.16413967 -0.19376355 -0.4436233   0.44620508  0.05850516
  -0.11386774 -0.3904782   0.08668881  0.15045048]
 [-0.19133005  0.41280606  0.15774411  0.09356532  0.09105669  0.20167015
   0.25692183 -0.4020199  -0.4337985   0.33612907]
 [ 0.3156307   0.11285669 -0.55082923  0.44292948  0.08078012 -0.02160936
   0.09686654  0.24624896 -0.30146688 -0.05306283]
 [ 0.35920274 -0.20268211  0.3080976   0.03791938 -0.10790671 -0.02458042
  -0.08566

# At the end are the final weights after the compiling was done that minimize the loss. Updating the biases in backpropagation is similar to updating the weights.

In [89]:
print("")
print('---------- Evaluation on Training Data ----------')
print("MSE: ", mean_squared_error(y_train, pred_train))
print("")


---------- Evaluation on Training Data ----------
MSE:  0.007889426980954031



In [90]:
print('---------- Evaluation on Test Data ----------')
print("MSE: ", mean_squared_error(y_test, pred_test))
print("")

---------- Evaluation on Test Data ----------
MSE:  0.009030063781459709



In [91]:
from sklearn.metrics import r2_score

In [92]:
r2_score(y_train, pred_train)

0.7544909036044207

In [93]:
r2_score(y_test, pred_test)

0.7094644710495845

# The model is giving good results above 70% and is not overfitted. 

Let’s now plot the results on a chart and compare actual and predicted values. Note, we use the inverse_transform function to convert targets and predictions from scaled (we used MinMaxScaler before training RNN) to the original value range.

In [94]:
#97 days to check the median temperature. 
print(len(y_test))
print(len(pred_test))

55
55


In [95]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(range(0,len(y_test))),
                         y=scaler.inverse_transform(y_test).flatten(),
                         mode='lines',
                         name='Median Temperature - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.add_trace(go.Scatter(x=np.array(range(0,len(pred_test))),
                         y=scaler.inverse_transform(pred_test).flatten(),
                         mode='lines',
                         name='Median Temperature - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Observation'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Median Daily Temperatures in Sydney", 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

fig.show()

# The above results are for the test dataset. The prediction looks pretty accurate, but remember that we take 12 prior data points in each case and only predict the next one. Therefore, the results of this specific model would be a lot less accurate if we tried to predict multiple points into the future, as I will show in a later example.

# With the current setup, we feed in 12 days worth of data and get the prediction for the next day
# We want to create an array that contains 12-day chunks offset by one day at a time.

# You will recall that we had every 13th observation in a sequence as our target during the training and prediction of the above model. But what if we wanted to use the model to generate predictions for every item (day) in our dataframe. The following code does exactly that:

# Below is the way how we can make a prediction for every day in the data instead of every 12 days. 

In [96]:
# Select only Canberra 
#dfSydney=df[df['Location']=='Sydney'].copy()

In [97]:
X_every= dfSydney[['MedTemp']]

In [98]:
X_every

,MedTemp
30176,20.95
30177,22.55
30178,23.05
30179,21.50
30180,22.70
...,...
33515,14.10
33516,14.25
33517,13.55
33518,14.70


In [99]:
X_every=scaler.transform(X_every)
X_every

array([[0.47433265],
       [0.54004107],
       [0.56057495],
       ...,
       [0.17043121],
       [0.21765914],
       [0.16632444]])

In [100]:
X_every.shape

(3339, 1)

In [101]:
time_step

12

In [102]:
len(X_every)-time_step

3327

In [103]:
for i in range(0, len(X_every)-time_step):
    if i==0:
        #the first seven values do not have predictions. 
        X_comb=X_every[i:i+time_step]
    else: 
        X_comb=np.append(X_comb, X_every[i:i+time_step])
X_comb=np.reshape(X_comb, (math.floor(len(X_comb)/time_step), time_step, 1))
print(X_comb.shape)

(3327, 12, 1)


In [104]:
X_comb[0]

array([[0.47433265],
       [0.54004107],
       [0.56057495],
       [0.49691992],
       [0.54620123],
       [0.58726899],
       [0.53593429],
       [0.42505133],
       [0.37782341],
       [0.41067762],
       [0.44147844],
       [0.56262834]])

In [105]:
len(X_comb[0])

12

In [106]:
# Use the reshaped data to make predictions and add back into the dataframe
# np.zeros(time_step) - Set the first 7 numbers to 0 as we do not have data to predict
dfSydney['MedTemp_prediction'] = np.append(np.zeros(time_step), scaler.inverse_transform(model.predict(X_comb)))

In [107]:
dfSydney.head(20)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,MedTemp,MedTemp_prediction
30176,2008-02-01,Sydney,19.5,22.4,15.6,6.2,0.0,NaN,NaN,S,SSW,17.0,20.0,92.0,84.0,1017.6,1017.4,8.0,8.0,20.7,20.9,Yes,Yes,20.95,0.000000
30177,2008-02-02,Sydney,19.5,25.6,6.0,3.4,2.7,NaN,NaN,W,E,9.0,13.0,83.0,73.0,1017.9,1016.4,7.0,7.0,22.4,24.8,Yes,Yes,22.55,0.000000
30178,2008-02-03,Sydney,21.6,24.5,6.6,2.4,0.1,NaN,NaN,ESE,ESE,17.0,2.0,88.0,86.0,1016.7,1015.6,7.0,8.0,23.5,23.0,Yes,Yes,23.05,0.000000
30179,2008-02-04,Sydney,20.2,22.8,18.8,2.2,0.0,NaN,NaN,NNE,E,22.0,20.0,83.0,90.0,1014.2,1011.8,8.0,8.0,21.4,20.9,Yes,Yes,21.50,0.000000
30180,2008-02-05,Sydney,19.7,25.7,77.4,NaN,0.0,NaN,NaN,NNE,W,11.0,6.0,88.0,74.0,1008.3,1004.8,8.0,8.0,22.5,25.5,Yes,Yes,22.70,0.000000
30181,2008-02-06,Sydney,20.2,27.2,1.6,2.6,8.6,NaN,NaN,W,ENE,9.0,22.0,69.0,62.0,1002.7,998.6,6.0,6.0,23.8,26.0,Yes,Yes,23.70,0.000000
30182,2008-02-07,Sydney,18.6,26.3,6.2,5.2,5.2,NaN,NaN,W,S,15.0,15.0,75.0,80.0,999.0,1000.3,4.0,7.0,21.7,22.3,Yes,Yes,22.45,0.000000
30183,2008-02-08,Sydney,17.2,22.3,27.6,5.8,2.1,NaN,NaN,S,SE,7.0,15.0,77.0,61.0,1008.3,1007.4,7.0,8.0,18.9,21.1,Yes,Yes,19.75,0.000000
30184,2008-02-09,Sydney,16.4,20.8,12.6,4.8,3.0,NaN,NaN,SSW,W,19.0,9.0,92.0,91.0,1006.4,1007.6,7.0,7.0,17.1,16.5,Yes,Yes,18.60,0.000000
30185,2008-02-10,Sydney,14.6,24.2,8.8,4.4,10.1,NaN,NaN,W,SSE,11.0,20.0,80.0,53.0,1014.0,1013.4,4.0,2.0,17.2,23.3,Yes,No,19.40,0.000000


In [73]:
len(dfSydney), X_comb.shape, pred_test.shape, pred_train.shape

(3339, (3327, 12, 1), (55, 1), (222, 1))

In [72]:
3339 - 12

3327

# Check the differences in shape: dfSydney has all the values. X_comb does not have the first 12 values and pred_train, pred_test have values every sequence of 12 values. 

# Set the first 12 numbers to 0 as we do not have data to predict. Afterwards we have data to predict every day based on the previous 12 days and not every 12 days. 

In [108]:
#Set the first 12 numbers to 0 as we do not have data to predict
dfSydney['MedTemp_prediction'].head(12)

30176    0.0
30177    0.0
30178    0.0
30179    0.0
30180    0.0
30181    0.0
30182    0.0
30183    0.0
30184    0.0
30185    0.0
30186    0.0
30187    0.0
Name: MedTemp_prediction, dtype: float64

In [109]:

fig = go.Figure()
fig.add_trace(go.Scatter(x= dfSydney['Date'],
                         y= dfSydney['MedTemp'],
                         mode='lines',
                         name='Median Temperature - Actual',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.add_trace(go.Scatter(x= dfSydney['Date'],
                         y= dfSydney['MedTemp_prediction'],
                         mode='lines',
                         name='Median Temperature - Predicted',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Observation'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Median Daily Temperatures in Sydney", 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

fig.show()
#black is the actual median day and predicted are with color red. 

# we have a red line in 0 and then it goes up in date 2008-02-14 the 13th value of the dataframe. 

# 2008-02-14 is the first day that we start to have data that it is not 0. 

# Again, pretty decent result keeping in mind that we only predict temperature for one day ahead, not every 12 days for  Sydney.

What if we tried to predict temperatures for the next 365 days generating predictions for one day at a time? We will attempt this by iteratively adding new predictions to our 12-day sequence while at the same time dropping the oldest one from the sequence.

In [110]:
inputs=X_comb[-1:]
inputs

array([[[0.19301848],
        [0.2073922 ],
        [0.24640657],
        [0.21355236],
        [0.27720739],
        [0.21560575],
        [0.21971253],
        [0.25667351],
        [0.19301848],
        [0.19917864],
        [0.17043121],
        [0.21765914]]])

In [111]:
#last day of the dataframe. 
dfSydney[-1:]

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,MedTemp,MedTemp_prediction
33519,2017-06-25,Sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,W,13.0,13.0,73.0,32.0,1018.6,1015.4,1.0,1.0,9.4,18.8,No,No,13.45,14.582573


In [112]:
# Create empty list
pred_list = []

# Loop 365 times to create predictions for the next year
for i in range(365):   
    pred_list.append(list(model.predict(inputs)[0])) # Generate prediction and add it to the list
    inputs = np.append(inputs[:,1:,:],[[pred_list[i]]],axis=1) # Drop oldest and append latest prediction

# Create a dataframe containing 365 days starting 2017-06-26
newdf=pd.DataFrame(pd.date_range(start='2017-06-26', periods=365, freq='D'), columns=['Date'])

# Add 365 days of model prediction from previous step
newdf['MedTemp_prediction']=scaler.inverse_transform(pred_list)

# Concatenate the orginal dataframe containing Canberra data and the new one containing predictions for the next 365 days
dfSydney2=pd.concat([dfSydney, newdf], ignore_index=False, axis=0, sort=False)

# Finally, we reuse the chart plotting code from the previous step to show the results for the past two years + the prediction for the next 365 days.

In [113]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= dfSydney2['Date'][-730:],
                         y= dfSydney2['MedTemp'][-730:],
                         mode='lines',
                         name='Median Temperature - Actual',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.add_trace(go.Scatter(x= dfSydney2['Date'][-730:],
                         y= dfSydney2['MedTemp_prediction'][-730:],
                         mode='lines',
                         name='Median Temperature - Predicted',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Observation'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Degrees Celsius'
                )

# Set figure title
fig.update_layout(title=dict(text="Median Daily Temperatures in Canberra", 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

fig.show()
#black is the actual median day and predicted are with color red. 

# And we can see that using the existing RNN model for anything longer than day+1 prediction is not wise. The reasons for such results are that we designed it only to predict one day ahead and partially influenced by RNNs having a relatively “short memory.”

----------------------------------------------------------------------------------------------------------------------------